In [14]:
# We import jsonfiy from Flask
import pandas as pd
from flask import Flask, url_for, request, jsonify, json
from joblib import dump, load
from werkzeug.exceptions import HTTPException
import re

main_path = '/Users/admin/Jedha/Fake_reviews_detection/1_Training_models/text_only_models/'
#
svm = load(main_path + '/main_model.pkl')
#
preprocessor = load(main_path + '/preprocessor.pkl')
#
text_vectorizer = load(main_path + '/text_vectorizer.pkl')
#
topic_extractor = load(main_path + '/topic_extractor.pkl')


In [25]:
review = review.replace('\n', ' ')

length = len(review)
exclam_count = len(''.join(ch for ch in review if ch =='!'))
uppercase_word_count = sum(map(str.isupper, review.split()))

review_clean = re.sub(r"[^A-zÀ-ÿ0-9' ]+", " ", review).lower()
review_clean = re.sub(' +', ' ',review_clean).strip()





In [54]:
vectrorized_item = text_vectorizer.transform([review_clean])
vectrorized_item.toarray()

In [52]:
len(text_vectorizer.vocabulary_)

1752

In [50]:
topic = topic_extractor.transform(vect)

ValueError: dimension mismatch

In [7]:
cd '/Users/admin/Jedha/'

/Users/admin/Jedha


In [35]:


import tensorflow as tf
import tensorflow_datasets as tfds
import keras.backend as K
import pathlib 
import pandas as pd 
import numpy as np
from langdetect import detect
import seaborn as sns
import io 
import matplotlib.pyplot as plt
import boto3
from datetime import datetime as dt

from s3_credentials import *

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

import unidecode

session = boto3.Session(aws_access_key_id= YOUR_ACCESS_KEY, 
                        aws_secret_access_key= YOUR_SECRET_KEY)

s3 = session.resource("s3")
client = session.client("s3")

# full_dataset_reworked
obj = s3.Object('jedha-fake-reviews-project', "datasets/full_dataset.csv")
dataset = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), low_memory = False, index_col=0)

#_____________________________________________________________________
######### Cleaning the dataset and adding new columns #########
#_____________________________________________________________________

#we drop rows in which restaurant infos are not available (miss scraped)
dataset = dataset.dropna(subset = ['restaurant_average_rating', 'restaurant_reviews_count', 'restaurant_expensiveness', 'restaurant_name'])

#adding a column with the length of the text review
dataset['text_length'] = dataset['text_review'].apply(lambda x : len(x))

#_____________________________________________________________________
######### Fixing existing columns values and types #########
#_____________________________________________________________________

#for the user_total_image_posted column, if user_total_image_posted is NA it means there is there's no image
    # so we set the value to 0
dataset.loc[dataset['user_total_image_posted'].isna(), 'user_total_image_posted'] = 0

#for the date column,  there is some miss scraps that we want to fix
    # a correct data must have a length of 10 , if it is smaller than 10 it's becasue we scrapped the number of images of the user instead
    # we may have to scrap again those lines to fix it
    # we keep only the rows where the date is correct 
mask_not_date = dataset['date'].apply(lambda x: len(x)) < 10
dataset = dataset.loc[mask_not_date == False, :]
    # if te length is greater than 10 is it is beacause we scraped the date + somme additional words ('Avis mis à jour') so we will keep only the part with the date
mask_date_to_fix = dataset['date'].apply(lambda x: len(x)) > 10
dataset.loc[mask_date_to_fix, 'date' ] = dataset.loc[mask_date_to_fix, 'date' ].str.split('\n').str[0]
    #finally we can convert the date column to a datetime format
dataset['date'] = pd.to_datetime(dataset['date'])

#for the language column,
    # We set all languages other than fr and eu to other to avoiding having lots of categories (causing troubles with OneHotEncoder)
mask_is_not_fr_en = (dataset['language'] != 'fr') & (dataset['language'] != 'en')
dataset.loc[mask_is_not_fr_en ,'language'] = 'other'

#for the username column,
    # Setting Not Yelp User = 'Not_Yelp_User
dataset.loc[dataset['username'] == 'Not Yelp User', 'username' ] = 'Not_Yelp_User'

#for the photos_for_review column, 
    # value -1 is in fact 0 (no photos found by the scraper)
dataset.loc[dataset['photos_for_review'] == '-1.0', 'photos_for_review' ] = '0'
    # value L is in fact 0 (no photos found by the scraper but scraped the first letter of "L'avis du jour" which happens when the reviews was updated by the user)
dataset.loc[dataset['photos_for_review'] == 'L', 'photos_for_review' ] = '0'
    # finally we can convert the photos_for_review column to an int format
dataset['photos_for_review'] = dataset['photos_for_review'].astype('int')

#for the photos_for_review column, 
    # when there's no info about the expensiveness we set it to -1
dataset.loc[dataset['restaurant_expensiveness'] == 'N/C', 'restaurant_expensiveness']  = -1
    # we can convert the restaurant_expensiveness column to an int format
dataset['restaurant_expensiveness'] = dataset['restaurant_expensiveness'].astype('int')

#for the is_real_review column, 
    # when reverse the values 0 and 1 and rename the column is_fake_review it will make our work easier with sklearn features
dataset['is_real_review'] = dataset['is_real_review'].apply(lambda x: 1 if x == 0 else 0)
dataset.rename(columns={'is_real_review': 'is_fake_review'}, inplace=True)



In [36]:
french_reviews = dataset.loc[dataset['language'] =='fr',['text_review', 'is_fake_review']].reset_index(drop = True)

In [10]:
review = french_reviews['text_review'][0]

In [ ]:
@app.route("/predict", methods=["POST"])
def predict():
    # Check parameters
    if request.json:
        # Get JSON as dictionnary
        json_input = request.get_json()
        value = [[json_input['year']]]
        prediction = regressor.predict(value)
        
        # Return prediction
        response = {
            # Since prediction is a float and jsonify function can't handle
            # floats we need to convert it to string
            "prediction": str(prediction),
        }
        return jsonify(response)


if __name__ == "__main__":
    app.run(debug=True)

In [42]:
fr_reviews = pd.read_csv('/Users/admin/Downloads/dataset_clean.csv', index_col = 0)

In [47]:
fr_sample = fr_reviews['text_review_clean'].sample(100).reset_index(drop = True)

In [51]:
len(vectorizer.vocabulary_)

NameError: name 'vectorizer' is not defined